<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера V2</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Привет ещё раз!

Не все оставленые мной сейчас новые комментарии зелёные, но критического ничего нет, смело принимаю твою работу.

Успехов с последующими проектами!

<div class="alert alert-info">
<font size="4"><b>Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
    Привет, Станислав! Спасибо, что прислал задание :) <br>Меня зовут Никита Сон и я буду проверять твой проект. Предлагаю обращаться друг к другу на ты, как это принято в Практикуме, если ты не против. Но если хочешь на Вы - не буду возражать 🙂
        
Поехали 🚀
    <br />

Мои комментарии обозначены пометкой <b>Комментарий ревьюера</b>. При внесении правок в проект, пожалуйста, не меняй и не удаляй их т.к. дальнейшая проверка будет происходить в том числе на основе того, исправлены замечания в комментариях или нет. Зато ты всегда можешь оставить свои комментарии для меня :)
    </font>
</div>

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
В зелёных блоках будут позитивные комментарии
<br />
    </font>
</div>

<div class="alert alert-warning">
<font size="4"><b>⚠️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
В жёлтых - некритичные замечания
    </font>
</div>

<div class="alert alert-danger">
<font size="4"><b>❌ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br /> 
В красных - важные замечания, которые надо обязательно устранить
    </font>
</div>

<div class="alert alert-info">
<font size="4">🍕<b> Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br /> 
В синих с пиццей - различные нейтральные сообщения, советы на будущее и прочее.
    </font>
</div>

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

### Подключение библиотек

In [1]:
! pip install phik -q
#! pip install shap -q
#! pip install scikit-learn==1.2.2 -q

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from phik import phik_matrix
from phik.report import plot_correlation_matrix

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = np.random.RandomState(12345)

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />    
Сразу задать константу с рандом стейтом - отличное решение

### Функции вывода графиков

In [3]:
def plots_hist_box(df):
    numeric_cols = df.select_dtypes(include=['number']).columns

    num_plots = len(numeric_cols)
    plt.figure(figsize=(15, 5*num_plots))
    
    for i, col in enumerate(numeric_cols):
        plt.subplot(num_plots, 2, 2*i + 1)
        if len(df[col].unique()) > 15:
            df[col].hist(bins=15)
        else:
            sns.countplot(df[col])
        plt.title(f'Распределение {col}')
        plt.xlabel(col)
        plt.ylabel('Количество значений')

        plt.subplot(num_plots, 2, 2*i + 2)
        df.boxplot(column=col)
        plt.title(f'Распределение {col}')

    plt.tight_layout()
    plt.show()
    
def plots_pie(df):
    categorical_cols = df.select_dtypes(exclude=['number']).columns

    num_plots = len(categorical_cols)
    plt.figure(figsize=(12, 5 * num_plots))

    for i, col in enumerate(categorical_cols):
        plt.subplot(num_plots, 1, i + 1)
        plt.pie(df[col].value_counts(), autopct = '%1.1f%%', labels=df[col].sort_values().unique())
        plt.title(f'Соотношение {col}')
        plt.xlabel(col)
        plt.ylabel('Количество')

    plt.tight_layout()
    plt.show()

### Загрузка данных

In [4]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_0.info()
display(geo_data_0.head())

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/geo_data_0.csv'

In [ ]:
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_1.info()
display(geo_data_1.head())

In [ ]:
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')
geo_data_2.info()
display(geo_data_2.head())

### Проверка на наличие пропусков

In [ ]:
geo_data_0.isna().sum()

In [ ]:
geo_data_1.isna().sum()

In [ ]:
geo_data_2.isna().sum()

Пропусков не обнаружено

### Анализ данных

In [ ]:
plots_hist_box(geo_data_0)

In [ ]:
plots_hist_box(geo_data_1)

In [ ]:
plots_hist_box(geo_data_2)

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Я ревьюер простой: вижу гистограммы и боксплоты признаков - пишу зелёный комментарий

В целом данные нормально распределены или близки к нормальному распределению по параметрам. Анализируя графики, складывается ощущение что датасет для второго региона имеет сильные искажения или ошибки в данных. Больше всего это сказывается на параметрах f2 и product. <br><br>
Говоря о датасетах 1-ого и 3-его регионах, стоит заметить что сильных выбросов в данных не обнаружено и данные выглядят хорошо.<br><br>
Подробнее рассмотрим product.

### Распределение объема скважин

In [ ]:
plt.figure(figsize=(15,8))
geo_data_0['product'].hist(bins=175)
plt.xlabel('Объем, тыс. барелей')
plt.ylabel('Количество скважин')
plt.title('Распределение объема скважин первого региона')
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
geo_data_1['product'].hist(bins=90)
plt.xlabel('Объем, тыс. барелей')
plt.ylabel('Количество скважин')
plt.title('Распределение объема скважин второго региона')
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
geo_data_2['product'].hist(bins=180)
plt.xlabel('Объем, тыс. барелей')
plt.ylabel('Количество скважин')
plt.title('Распределение объема скважин третьего региона')
plt.show()

### Корреляционный анализ

Так как объем данных большой и считает долго, возьмен образец данных.

In [ ]:
plt.figure(figsize=(15, 8))

plt.subplot(3, 3, 1)
plt.title('коэффициент для первого региона phi k')
corr_matrix0 = geo_data_0.drop(columns=(['id'])).corr(method='spearman');
sns.heatmap(corr_matrix0,
            xticklabels=corr_matrix0.columns.values,
            yticklabels=corr_matrix0.columns.values,
           fmt=".2f", annot=True, linewidth=.5);

plt.subplot(3, 3, 2)
plt.title('коэффициент для второго региона phi k')
corr_matrix1 = geo_data_1.drop(columns=(['id'])).corr(method='spearman');
sns.heatmap(corr_matrix1,
            xticklabels=corr_matrix1.columns.values,
            yticklabels=corr_matrix1.columns.values,
           fmt=".2f", annot=True, linewidth=.5);

plt.subplot(3, 3, 3)
plt.title('коэффициент для третьего региона phi k')
corr_matrix2 = geo_data_2.drop(columns=(['id'])).corr(method='spearman');
sns.heatmap(corr_matrix2,
            xticklabels=corr_matrix2.columns.values,
            yticklabels=corr_matrix2.columns.values,
           fmt=".2f", annot=True, linewidth=.5);
plt.tight_layout()
plt.show()

<s>Великолепная корреляция между целевым признаком и параметрами f0, f1, f2 в первом и третьем регионе.</s>

<div class="alert alert-danger">
<font size="4"><b>❌ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
        <b></b>

Считать корреляции по 1% данных - занятие очень сомнительное, ведь реальные значения коэффициентов могут слишком сильно отличаться от того, что ты получишь (и так оно и есть в твоём случае). Так что выражаю ноту протеста. Если же ты действительно считаешь, что ждать слишком долго, ничто не мешает тебе вместо коэффициентов фик вычислить просто корреляцию Спирмена, она вычисляется сильно быстрее.

<div class="alert alert-info">
<h2> Комментарий студента <a class="tocSkip"> </h2>
Так и сделал, пересчитал и сделал выводы.
</div>

Судя по матрицам корреляции параметр f2 имеет заметную корреляцию с целевым признаком в первом и третьем регионе. (Второй не смотрим, он подозрительный) 

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера V2</b></font>
    <br /> 
    <font size="3", color = "black">
<br />    
Другое дело :)

### Вывод:
Данные готовы к обучению и проверки модели.<br><br>

Во втором датасете обнаружено много месторождений с нулевым объемом, и все варианты значений укладывается в 12 значений. Сравнив распределение признаков с первым и третьим регионом то можно сказать что в данных какая-то ошибка либо они повреждены.

<div class="alert alert-warning">
<font size="4"><b>⚠️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br /> 
В целом, анализ весьма неплохой, но проверки на дубликаты ему не хватает
    </font>
</div>

## Обучение и проверка модели

### Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.

Колонка id не информативна и не будет участвовать в обучении модели. Удалим ее из каждого датасета.

In [ ]:
geo_data_0.drop('id',inplace=True,axis=1)
geo_data_0.info()
geo_data_1.drop('id',inplace=True,axis=1)
geo_data_1.info()
geo_data_2.drop('id',inplace=True,axis=1)
geo_data_2.info()

Добавим функцию разбиения датасета на обучающую и валидационную выборки

In [ ]:
def data_split(data, target, valid_size):
    # data - dataframe
    # target - target feature
    # valid_size - train and valid proportion
    features = data.drop([target] , axis=1)
    target = data[target]

    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=valid_size, random_state=RANDOM_STATE)
    
    return features_train, features_valid, target_train, target_valid

Категориальные признаки в даннных отсутствуют, имеются только количественные. Отмасштабируем их перед обучением модели. Для этого добавим функцию.

In [ ]:
def data_split_scale(data, target, valid_size, cols_to_scale):
    features_train, features_valid, target_train, target_valid = data_split(data, target, valid_size)
    
    scaler = StandardScaler()
    scaler.fit(features_train[cols_to_scale])
    features_train_scaled = scaler.transform(features_train[cols_to_scale])
    features_valid_scaled = scaler.transform(features_valid[cols_to_scale])
    return features_train_scaled, features_valid_scaled, target_train, target_valid

Объявим функцию, которая на выходе выдаст нам валидационные и предсказанные значения

In [ ]:
def make_model_get_result(data, target, valid_size, cols_to_scale):
    train_x, valid_x, train_y, valid_y = data_split_scale(data, target, valid_size, cols_to_scale)
    model = LinearRegression()
    model.fit(train_x, train_y)
    predict_y = model.predict(valid_x)
    return valid_y, predict_y

### Обучите модель и сделайте предсказания на валидационной выборке.

Далее работать будем с моделью линейной регрессии.

In [ ]:
# Модель и предсказания для первого региона 
geo0_valid_y, geo0_predict_y = make_model_get_result(geo_data_0, 'product', 0.25, ['f0','f1','f2'])

In [ ]:
# Модель и предсказания для второго региона
geo1_valid_y, geo1_predict_y = make_model_get_result(geo_data_1, 'product', 0.25, ['f0','f1','f2'])

In [ ]:
# Модель и предсказания для второго региона
geo2_valid_y, geo2_predict_y = make_model_get_result(geo_data_2, 'product', 0.25, ['f0','f1','f2'])

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
С одной стороны, ты, безусловно, молодец, что оформляешь код в функции. Это в целом здравый подход, а уж когда почти вся работа - повторение однотипных шагов над несколькими разными датасетами - это тем более разумно.

<div class="alert alert-info">
<font size="4">🍕<b> Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
С другой, почему бы обучение модели не сделать частью одной большой функции, которой ты передаёшь аргументом датасет, она его обрабатывает, обучает модель, которую и выдаёт? Кажется, было бы удобнее.

<div class="alert alert-danger">
<font size="4"><b>❌ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
        <b></b>

А кроме того, что это было бы удобнее, это бы, очень вероятно, уберегло бы тебя от ошибки. Обрати внимание на то, как ты получаешь предсказания модели для региона под индексом 1: точно ли ты используешь правильную переменную с моделью, когда вызываешь `predict`?

<div class="alert alert-info">
<h2> Комментарий студента <a class="tocSkip"> </h2>

Ой. Проглядел ошибку. Переоформлю в отдельную функцию.
</div>

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера V2</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
NICE JOB :)

### Сохраните предсказания и правильные ответы на валидационной выборке.

In [ ]:
results_geo0 = pd.DataFrame({'real' : geo0_valid_y, 'predict' : geo0_predict_y})
results_geo1 = pd.DataFrame({'real' : geo1_valid_y, 'predict' : geo1_predict_y})
results_geo2 = pd.DataFrame({'real' : geo2_valid_y, 'predict' : geo2_predict_y})

display(results_geo0.head())
display(results_geo1.head())
display(results_geo2.head())

### Напечатайте на экране средний запас предсказанного сырья и RMSE модели.

In [ ]:
total_result = pd.DataFrame({'Параметр':['Сред. запас','Действ. общий запас','Предсказ. общий запас','RMSE'],
                             'Регион 1':[round(results_geo0.predict.mean(),3),
                                         round(results_geo0.real.sum(),3),
                                         round(results_geo0.predict.sum(),3),
                                         round(mean_squared_error(results_geo0.real, results_geo0.predict)**0.5,3)],
                             'Регион 2':[round(results_geo1.predict.mean(),3),
                                         round(results_geo1.real.sum(),3),
                                         round(results_geo1.predict.sum(),3),
                                         round(mean_squared_error(results_geo1.real, results_geo1.predict)**0.5,3)],
                             'Регион 3':[round(results_geo2.predict.mean(),3),
                                         round(results_geo2.real.sum(),3),
                                         round(results_geo2.predict.sum(),3),
                                         round(mean_squared_error(results_geo2.real, results_geo2.predict)**0.5,3)]
                            })

display(total_result)

### Проанализируйте результаты.

Для того чтобы понять, насколько хорошо справилась модель, учтем как метрику, так и общие запасы сырья в регионе. <br><br>
Если обратиться к метрике RMSE то наилучшим образом модель линейной регрессии обучилась на данных второго региона, а корень из среднеквадратичной ошибки составил - 0.893, далее по качеству идет первый и третий регион с значениями RMSE 37.579 и 40.030 соответственно.
<br><br>
Однако, если чуть отойти от задания и глянуть насколько точно модель предсказала общий запас сырья в регионах, то увидим, что ближе всего к реальности получилось в первом регионе, далее третий и замыкает второй. Вцелом ничего удивительного, так как в датасете по второму региону много пропусков в записях о количестве сырья в месторождениях. 
<br><br>
Также следует отметить что средний запас предсказанного сырья почти одинаковый для каждого региона.

### Вывод:

Исходя из проведенного анализа и обучения и проверки модели, можно сделать вывод что перспективными для выбора под разработку являются первый и третий регионы. Данные второго региона аномальны и, наверняка, содержат ошибку. С текущими данными по второму региону рискованно делать его приорететным вариантом для проведения разработки и добычи.
<br><br>
Говоря о модели, можно сказать что результат получился удовлетворительным.

## Подготовка к расчёту прибыли

### Все ключевые значения для расчётов сохраните в отдельных переменных.

Все ключевые моменты мы зафексировани в табличке, вспомним, что там получилось

In [ ]:
display(total_result)

Бюджет на разработку скважин в регионе — 10 млрд рублей.<br><br>
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.<br><br>
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.<br><br>
Далее деньги будем считать в миллионах рублей, чтобы не путаться в нулях.

In [ ]:
BUDGET = 10e+03 # Бюджет на разработку скважин в регионе 
RESEARCH_FIELDS = 500 # Точек для исследования
SELECTED_FIELDS = 200 # Точек для разработки
PRODUCT_PRICE = 0.45 # Цена за тыс. барелей

<div class="alert alert-info">
<font size="4">🍕<b> Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
По стандарту разработки PEP8 константы принято именовать КАПСОМ. Например, так:

`BUDGET = 10e+03`

<div class="alert alert-info">
<h2> Комментарий студента <a class="tocSkip"> </h2>

Поправил
</div>

### Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе. 

А сколько в среднем должен быть запас месторождения, чтобы выйти в 0?

In [ ]:
print('Средней объем месторождения для безубыточной добычи:', round((BUDGET/SELECTED_FIELDS)/PRODUCT_PRICE,3), 'тыс. баррелей')

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Расчёт корректный

Так, а сколько всего нужно добыть в регионе чтобы выйти в ноль? 

In [ ]:
print('Средней объем месторождения для безубыточной добычи:', round(BUDGET/PRODUCT_PRICE,3), 'тыс. баррелей')

Чуть больше 22 миллионов баррелей, много.

### Напишите выводы по этапу подготовки расчёта прибыли.

Сравнив данные видно, что среднего запаса скважин недостаточно чтобы выйти в окупаемость. Поэтому нам предстоит выбрать только самые большие скважины.

## Расчёт прибыли и рисков 

### Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели.

In [ ]:
def fields_profit(target, predict):
    # Выберите скважины с максимальными значениями предсказаний. 
    best = predict.sort_values(ascending=False).head(SELECTED_FIELDS)    
    #Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
    total_profit = target.loc[best.index].sum()
    #Рассчитайте прибыль для полученного объёма сырья.
    return (total_profit*PRODUCT_PRICE) - BUDGET

## Посчитайте риски и прибыль для каждого региона:

### Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

In [ ]:
# Функция применяет технику Bootstrap согласно заданию, а возвращает 
# среднюю прибыль, квантиль 2,5% и 97,5%, а также процент риска. Возвращаемые значения выводятся на экран
def bootstrap_1000(target, predict):
    values=[]
    lost = 0
    for i in range(1000):
        target_sample = target.sample(random_state=RANDOM_STATE, n=RESEARCH_FIELDS, replace=True)
        predict_sample = predict[target_sample.index]
        profit = fields_profit(target, predict_sample)
        #print(profit)
        if profit < 0:
            lost +=1
        values.append(profit)
    
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    average = values.mean()
    risk = lost / 1000
    print('Средняя прибыль по региону:',round(average,3),'млн. рублей')
    print('95%-й доверительный интервал:',round(lower,3),'-',round(upper,3),'млн. рублей')
    print('Процент риска',round(risk*100,2),'%')
    sns.distplot(values)
    return average, lower, upper, risk

<div class="alert alert-info">
<font size="4">🍕<b> Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
        <b></b>

Раз уж ты чуть ранее объявил константу `RANDOM_STATE` (в чём ты, безусловно, молодец), то почему бы не сделать `state = np.random.RandomState(RANDOM_STATE)`?

<div class="alert alert-danger">
<font size="4"><b>❌ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
С одной стороны мелочь, но я к ней очень строг.

В чём разница того, что было в тренажёре и того, что тут в проекте? Там была задача классификации, а тут - регрессии. Поэтому там предсказания модели были вероятностями принадлежности к классам, а у нас - количество продукта в скважине. Никакого отношения прогноз нашей модели к вероятностям не имеет, поэтому переменных с названиями probabilities или probs быть не должно. Имена переменных должны соответствовать их содержимому.

<div class="alert alert-info">
<h2> Комментарий студента <a class="tocSkip"> </h2>

Понял, исправил!
Оказывается если RANDOM_STATE указать просто числом а не np.random.RandomState(12345) то sample постоянно выдает один и тот же набор данных.
</div>

<div class="alert alert-info">
<font size="4">🍕<b> Комментарий ревьюера V2</b></font>
    <br /> 
    <font size="3", color = "black">
        <b></b>

Именно так, потому ТЗ проекта и просит использовать np.random.RandomState в сэмплировании :)

<div class="alert alert-warning">
<font size="4"><b>⚠️ Комментарий ревьюера V2</b></font>
    <br /> 
    <font size="3", color = "black">
        <b></b>

Не очень понимаю зачем тебе разделение единого датафрейма `results_geo` на предсказания и таргеты? Чтобы сделать сэмпл вызываешь `results_geo.sample(...)`, у тебя получается кусок датафрейма, в котором есть и предсказания и таргеты, передаёшь его в вычисление прибыли, там сортируешь по одному столбцу, берёшь значения из другого. Когда у тебя уже есть единый датафрейм, то намного удобнее оперировать им целиком, а не делить обратно на pd.Series

### Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

Применим функцию к полученным ранее данным

In [ ]:
plt.figure(figsize=(15,8))

print('Результаты по региону 1')
bootstrap_1000(results_geo0.real,results_geo0.predict)
print('\nРезультаты по региону 2')
bootstrap_1000(results_geo1.real,results_geo1.predict)
print('\nРезультаты по региону 3')
bootstrap_1000(results_geo2.real,results_geo2.predict)

plt.xlabel('Прибыль, млн. руб.')
plt.ylabel('Плотность распределения')
plt.legend(['1','2','3'],title="Регион")
plt.show()

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера V2</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
KPACUBO :)

<div class="alert alert-info">
<font size="4">🍕<b> Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br /> 
Было бы неплохо как-то визуализировать финальные результаты, они же то, ради чего всё и делается. Можно построить ящики с усами для выборочных прибылей, или же гистограммы, добавив на них границы найденных доверительных интервалов.

### Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

Лучший результат оказался во втором регионе, но как говорили ранее. Данные в датасете выглядят подозрительно, а также присутствуют множественные пропуски в колонке product.<br><br>
В итоге, первая модель предоставляет минимальный процент риска - 7.5%, что соответсвует и самую большую среднюю прибыль по региону в 380 млн. рублей.


## Вывод
В ходе работы были загруженны и подготовлены данные, обучены и проверены модели для каждого из трех регионов, проведена подготовка и рассчитаны риски и прибыль для регионов.<br><br>
Цели работы частично достигнуты, так как ни одна из моделей не смогла гарантировать риск равным или менее 2.5%. Но наилучшим вариантом для разработки является первый регион с наибольшей средней прибылью по региону.


<div class="alert alert-info">
<font size="4">🍕<b> Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br /> 
Время подвести итоги. В целом, работа оставила достаточно приятное впечатление, но, доработать немножко нужно:

+ Исправить маленькую ошибку в коде. После её устранения не забудь прогнать код и проверить выводы на актуальность: всё в работе может очень сильно поменяться
+ Пересчитать корреляции по всему датасету, не обязательно фик
+ Поправить имена переменных так, чтобы не было логического несоответствия

И на этом пока что всё. Жду новую отправку. Надеюсь и верю, что финальную :)

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован